In [7]:
import pandas as pd
import numpy as np
import pyscf
from pyscf import gto, scf, ao2mo

print(f"Pandas Version: {pd.__version__}, PySCF Version: {pyscf.__version__}, Numpy Version: {np.__version__}")


Pandas Version: 2.2.2, PySCF Version: 2.6.2, Numpy Version: 1.26.2


Currently This Script is only good for atoms but can be easily expanded.

This script Scraps a basis set from the Molpro website and then translates it into the PySCF internal format. Important notes: The molpro website is scrapped with pandas. whend pd.read_html returns data fields, each data field coresponds to one angular momentum. Each data field contains the basis set pre-exponential factor in the first column. Each column after the first column contain the coefficients of one contraction. 

With this information, one can convert the basis set into any format. PySCF currently (version 2.6.2) internally stores basis sets as follows:
$$\{'A': [[l_1, [\varepsilon_{11}, c_{11}], \dots, [\varepsilon_{1n}, c_{1n}],], [l_2,  [\varepsilon_{21} c_{21}], \dots, [\varepsilon_{2n} c_{2n}],]\dots [l_n, [\varepsilon_{n1}, c_{n1}], \dots, [\varepsilon_{nn}, c_{nn}],]\}
$$
Where '{}' indicates a dict and '[]' a list. One contraction is in
$$[l_1, [\varepsilon_{11}, c_{11}], \dots, [\varepsilon_{1n}, c_{1n}],]$$
where $l_1$ id the angular momentum as integer of one contraction and the following list contains each pre-exponential $\varepsilon_{11}$, combined with its contraction coefficient $c_{11}$.



In [12]:
element = 'K'
basis_type = 'aug-cc-pVQZ'
url = f"https://www.molpro.net/info/basis.php?search=1&element={element}&basis={basis_type}&print=1"
dfs = pd.read_html(url)

In [13]:
# minimum input for mol object
mol = gto.Mole()
mol.atom = f'''{element}  0 0 0'''
mol.charge = 0
mol.spin = 1
mol.unit = 'AU'


In [14]:
# parsing of df objects according to description
contraction = 0
basis = {}
basis[element] = []

for l in range(0,len(dfs)):# each data field being one angular momentum
    if len(dfs[l].columns) > 1:# special case on website as of Jul25, 2024: if the primitive fuunction is to be used, sometimes only the exponent is given 
        for column in range(1,len(dfs[l].columns)):# coumns are contractions
            basis[element].append([l])
            for line in np.ndarray.tolist(dfs[l].values):
                if line[column] > 0.0:
                    basis[element][contraction].append([line[0],line[column]])
            contraction+=1
    else:# special case as of Jul25, 2024: if the primitive fuunction is to be used, sometimes only the exponent is given 
        for line in np.ndarray.tolist(dfs[l].values):
            basis[element].append([l , [line[0], 1.0]])

    

mol.basis = basis 

#for contraction in basis[element]:
#    print(contraction)

In [11]:
mol.build()
hf = scf.ROHF(mol).run()

converged SCF energy = -598.509883870499
